# Dictionary keys are:

- `id`: integer, id #
- `image_id`: integer, image id #
- `category_id`: 1 for penguin, 2 for turtle
- `bbox`: list of integers representing the bounding box coordinates in Pascal VOC format [xmin, ymin, xmax, ymax]
- `area`: integer representing area of bounding box.
- `segmentation`: empty list; add segmentation masks if you'd like!
- `iscrowd`: integer 0 or 1; whether the instance is a crowd or individual. Not relevant to this particular use case, but is a necessary key for some models.

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.feature import hog
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV